# AUDIO CLASSIFICATION BASED ON 'MODALITY' USING CHROMA FEATURES

## PART 2 : MACHINE LEARNING & AUTOMATIC AUDIO CLASSIFICATION

#### This series of notebooks demonstrate a tutorial for classifying audio files based on the tonal concept of 'modality in music'. The concept of 'modality' exists in various tonal music traditions around the world. In this tutorial, we apply supervised learning based on chroma features.

There are 2 main steps for the audio classification: Feature Extraction and Classification. 
---

#### This second notebook includes the manual Feature Selection step and the whole Pipeline of the Machine Learning process for Audio Classification based on 'Modality'.

#### Feature Selection is applied manually in order to test and evaluate the effect of choosing different statistical features and using features from varying parts of the audio.

### Input : 
#### PICKLE file : 
which is the output of the first notebook in the series (Feature Extraction).


### Output :
#### CSV file : 
The output CSV file is generated after the manual Feature Selection step.

#### Parameters:


 - numBins(int) : number of bins in the HPCPvectors ([12,24,36,48])
 
 
 - modality : Name of the modality type specific for the music tradition.
 
            Classical Western Music / Jazz : Mode, chord-scale, scale
            Classical Turkish Music : Makam
            Classical North Indian Music : Rag, Raga
            Classical Arab-Andalusian Music : Tab
 
 
 - region(float) : select the first 'x*100'% of the audio files for analysis. The classification is performed using the features that are extracted locally only from specified region of the songs.
 

 - combined (boolean) : if combined = 1, classification is performed using the combination of features that are extracted both locally and globally. if combined = 0, only locally obtained features are used for classification. The local region is specified by partSong. Default = 1.
 

 - featureSet (str) : 'mean', 'std', 'all'.  The set of statistical features to be included for automatic classification. Default = 'all'

In [1]:
import os, sys, json, csv
import numpy as np
import matplotlib.pyplot as plt
from modalityEval import *

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


- In the cell below, define the parameters that are explained above.

In [2]:
### NAVIGATE TO THE DIRECTORY WHERE THE .PKL FILES ARE LOCATED

#dataDir = 'data/'
dataDir = '../andalusian-corpora-notebooks/'

### SPECIFY THE NUMBER OF BINS IN CHROMA VECTORS
numBins = 12

### DEFINE THE TYPE OF MODALITY PRESENT IN THE MUSIC TRADITION OF INTEREST
modality = 'tab'

### SELECT THE REGION OF THE SONG TO BE ANALYZED LOCALLY. (region = 1 is equivalent of using the Global Features)
### ONLY APPLICABLE FOR MAKAM TRADITION
region = 0.3


### CHOOSE THE FEATURE SET ('mean','std','all')
featureSet = ['mean','std','all']

### SET combined = 1 FOR PERFORMING CLASSIFICATION USING THE COMBINATION OF LOCAL AND GLOBAL FEATURES
###     combined = 0 FOR USING ONLY THE LOCAL FEATURES (except the case region = 1, which corresponds to global features)
### SET combined = 1 FOR ONLY MAKAM TRADITION
combined = 0

In [3]:
##FILENAME : 'Features_20makam_for(numBin)bins.pkl'
filename = 'extractedFeatures12fortabtradition.pkl'  

dataSet, modalitySet = readDataset(dataDir,filename, modality, region)

print('Modality types in this dataset:  \n')
print(modalitySet)

Modality types in this dataset:  

{16}


#### Generate CSV

The cell below generates the CSV files with the proper format for the Machine Learning step. 

----> X = Features,     Y = Class Labels (ground truth)

In [5]:
for features in featureSet:

    generateCSV(dataDir,dataSet,region,features, modality, combined)

### MACHINE LEARNING

In [7]:
#APPLICABLE FOR ALL TRADTITIONS
if combined == 0:
    attributes=['mean','std','all']

#ONLY APPLICABLE FOR MAKAM TRADITION
elif combined == 1:
    attributes=['meanLocal+stdGlobal','stdLocal+meanGlobal','meanLocal+meanGlobal','stdLocal+stdGlobal']

modalityType = modality + 'Type'    
list_accuracy=[]
for attribute in attributes:

    file = dataDir+'DataCSVforstage'+'_'+str(numBins)+'bins_'+attribute+'.csv'
    df = pd.read_csv(file)
    df.pop('name'); dataclass=df.pop(modalityType)
    X=df; Y=dataclass
    cm,acc,f = machineLearning(dataDir,X,Y,attribute,numBins)
    list_accuracy.append([cm,acc])

best_model = max(enumerate(list_accuracy))[1]
modalitySet = sorted(modalitySet)
plot_confusion_matrix(best_model[0],modalitySet,normalize=False)

ValueError: The number of classes has to be greater than one; got 1